# Įvadas į promptų inžineriją
Promptų inžinerija – tai procesas, kurio metu kuriami ir optimizuojami promptai natūralios kalbos apdorojimo užduotims. Tai apima tinkamų promptų parinkimą, jų parametrų derinimą ir veikimo vertinimą. Promptų inžinerija yra labai svarbi siekiant aukšto tikslumo ir efektyvumo NLP modeliuose. Šioje dalyje susipažinsime su pagrindiniais promptų inžinerijos principais, naudodami OpenAI modelius kaip pavyzdį.


### Užduotis 1: Tokenizavimas
Susipažinkite su tokenizavimu naudojant tiktoken – atviro kodo greitąjį tokenizatorių iš OpenAI  
Daugiau pavyzdžių rasite [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst).


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Užduotis 2: Patikrinkite Github Models rakto nustatymą

Paleiskite žemiau esantį kodą, kad įsitikintumėte, jog jūsų Github Models galutinis taškas yra tinkamai sukonfigūruotas. Šis kodas tiesiog išbando paprastą užklausą ir patikrina, ar gaunamas atsakymas. Įvedus `oh say can you see`, atsakymas turėtų būti panašus į `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Pratimas 3: Išgalvojimai
Išbandykite, kas nutinka, kai paprašote LLM pateikti atsakymus į užklausą apie temą, kuri galbūt neegzistuoja, arba apie temas, kurių jis gali nežinoti, nes jos buvo už jo išankstinio mokymo duomenų rinkinio ribų (naujesnės). Pažiūrėkite, kaip keičiasi atsakymas, jei pabandysite kitą užklausą arba kitą modelį.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Užduotis 4: Pagal instrukciją  
Naudokite kintamąjį "text" pagrindiniam turiniui nustatyti  
ir kintamąjį "prompt" nurodymui, susijusiam su tuo pagrindiniu turiniu, pateikti.

Čia prašome modelio apibendrinti tekstą antros klasės mokiniui


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Užduotis 5: Sudėtingas užklausos pavyzdys
Išbandykite užklausą, kurioje yra sistemos, vartotojo ir asistento žinutės  
Sistema nustato asistento kontekstą  
Vartotojo ir asistento žinutės sukuria daugiapakopį pokalbio kontekstą

Atkreipkite dėmesį, kad asistento asmenybė sistemos kontekste nustatyta kaip „sarkastiška“.  
Pabandykite naudoti kitokį asmenybės kontekstą. Arba išbandykite kitokią įvesties/išvesties žinučių seką


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Pratimas: Tyrinėkite savo intuiciją
Aukščiau pateikti pavyzdžiai parodo jums šablonus, kuriuos galite naudoti kurdami naujas užklausas (paprastas, sudėtingas, instrukcijas ir kt.) – pabandykite sukurti kitus pratimus, kad išbandytumėte kai kurias kitas idėjas, apie kurias kalbėjome, pavyzdžiui, pavyzdžius, užuominas ir dar daugiau.



---

**Atsakomybės atsisakymas**:  
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, atkreipkite dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Svarbiai informacijai rekomenduojame profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už bet kokius nesusipratimus ar neteisingą interpretavimą, kilusį naudojantis šiuo vertimu.
